In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from muxcnn.utils import load_params, load_img
#from muxcnn.hecnn_par import *
#from muxcnn.comparator import ApprRelu
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv

## Torch model

In [2]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [3]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [4]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# MuxConv

In [5]:
from time import time

In [6]:
fhe_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [18]:
t0 = time()
result = fhe_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
1.2791 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
5.0029 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
5.0710 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
9.6721 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
4.6888 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
1.4301 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
8.9452 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =( 8, 8,64, 4, 4,  8)
4.4187 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
1.3442 s
41.8744 s


In [19]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8151,  -3.1035,  -2.9026,  -9.4615,   4.3280,  -6.9753,
         -3.5617,  -8.8841,  -7.5631], dtype=torch.float64)